This notebook generates the paragraph about the microCT-scanning from logfiles of the scans.

In [ ]:
import platform
import os
import pandas
import glob

In [ ]:
from parsing_functions import *

In [ ]:
# Different locations if running either on Linux or Windows
if 'Linux' in platform.system():
    BasePath = os.path.join(os.path.sep, 'home', 'habi', 'research-storage-uct', 'Archiv_Tape')
elif 'Windows' in platform.system():
    BasePath = os.path.join('R:', os.sep)
Root = os.path.join(BasePath, os.sep, 'SomeFolder')
Root = 'logfiles'
print('We are loading all the data from the folder %s' % Root)

In [ ]:
# Make us a dataframe for saving all that we need
Data = pandas.DataFrame()

In [ ]:
# Get *all* log files
# Using os.walk is way faster than using recursive glob.glob, see DataWrangling.ipynb for details
# Not sorting the found logfiles is also making it quicker
Data['LogFile'] = [os.path.join(root, name)
                   for root, dirs, files in os.walk(Root)
                   for name in files
                   if name.endswith((".log"))]

In [ ]:
# See what we get
Data.sample(n=5)

In [ ]:
log=Data.LogFile[0]

In [ ]:
fulllog(log)

In [ ]:
scanner(log, verbose=True)

In [ ]:
controlsoftware(log, verbose=True)

In [ ]:
source(log, verbose=True)

In [ ]:
camera(log, verbose=True)

In [ ]:
voltage(log, verbose=True)

In [ ]:
current(log, verbose=True)

In [ ]:
whichfilter(log, verbose=True)

In [ ]:
fulllog?

In [ ]:
numproj(log, verbose=True)

In [ ]:
stacks(log, verbose=True)

In [ ]:
projectionsize(log)

In [ ]:
overlapscan(log, verbose=True)

In [ ]:
Data.sample(n=10)

In [ ]:
def threesixtyscan(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if '360 Rotation' in line:
                if verbose:
                    print(line)
                threesixty = line.split('=')[1]
                print(threesixty)
                if 'YES' in threesixty:
                    threesixty = True
                elif 'NO' in threesixty:
                    threesixty = False
    return(threesixty)

In [ ]:
threesixtyscan(log, verbose=True)

In [ ]:
rotationstep(log, verbose=True)

In [ ]:
exposure(log, verbose=True)

In [ ]:
averaging(log, verbose=True)

In [ ]:
duration(log, verbose=True)

In [ ]:
pixelsize(log, verbose=True)

In [ ]:
Data.sample(n=5)

In [ ]:
version(log, verbose=True)

In [ ]:
ringremoval(log, verbose=True)

In [ ]:
beamhardening(log, verbose=True)

In [ ]:
Data['Scanner'] = [scanner(log) for log in Data['LogFile']]
Data['Software'] = [controlsoftware(log) for log in Data['LogFile']]

In [ ]:
Data['Voxelsize'] = [pixelsize(log) for log in Data['LogFile']]
Data['Voxelsize_rounded'] = [pixelsize(log,rounded=True) for log in Data['LogFile']]

In [ ]:
Data['Source'] = [source(log) for log in Data['LogFile']]
Data['Camera'] = [camera(log) for log in Data['LogFile']]

In [ ]:
Data['Voltage'] = [voltage(log) for log in Data['LogFile']]
Data['Current'] = [current(log) for log in Data['LogFile']]
Data['Filter'] = [whichfilter(log) for log in Data['LogFile']]

In [ ]:
Data['Stacks'] = [stacks(log) for log in Data['LogFile']]
Data['NumProj'] = [numproj(log) for log in Data['LogFile']]
Data['ProjSize'] = [projectionsize(log) for log in Data['LogFile']]
Data['RotationStep'] = [rotationstep(log) for log in Data['LogFile']]
Data['Wide'] = [overlapscan(log) for log in Data.LogFile]

In [ ]:
Data['RingRemoval'] = [ringremoval(log) for log in Data['LogFile']]
Data['Beamhardening'] = [beamhardening(log) for log in Data['LogFile']]

In [ ]:
Data['Exposure'] = [exposure(log) for log in Data['LogFile']]
Data['Averaging'] = [averaging(log) for log in Data['LogFile']]

In [ ]:
Data['Duration'] = [duration(log) for log in Data['LogFile']]

In [ ]:
Data['Version'] = [version(log) for log in Data['LogFile']]

In [ ]:
Data.to_csv('ScanningDetails.csv')

In [ ]:
Data.sample(n=10)

In [ ]:
# Copy-paste this to wherever you want the data in Markdown
print(Data.to_markdown())

In [ ]:
STOPHERE==

----

My microct blurb from http://simp.ly/publish/NBhZhH

In [ ]:
print('Based on the %s log files in %s' % (len(Data), Root))

In [ ]:
" OR ".join(str(value) for value in Data.Scanner.unique())

In [ ]:
print('After $PREPARATION, the',
      len(Data),
      'samples were imaged on a Bruker',
      " OR ".join(str(value) for value in Data.Scanner.unique()),
      'high-resolution microtomography machine (Control software version',
      " OR ".join(str(value) for value in Data.Software.unique()) + 
      ', Bruker microCT, Kontich, Belgium).')

In [ ]:
print('The machine is equipped with a',
      " OR ".join(str(value) for value in Data.Source.unique()),
      'X-ray source and a',
      " OR ".join(str(value) for value in Data.Camera.unique()),
      'camera.')

In [ ]:
# if len(Data.Scanner.unique()) > 1:
#     print('more')

In [ ]:
print('The X-ray source was set to a tube voltage of', 
      " OR ".join(str(value) for value in Data.Voltage.unique()),
      'kV and a tube current of',
      " OR ".join(str(value) for value in Data.Current.unique()),
      'µA, the x-ray spectrum was', end=' ')
if Data.Filter.unique():
    print('filtered by', " OR ".join(str(value) for value in Data.Filter.unique()), end=' ')
else:
    print('not filtered', end=' ')
print('prior to incidence onto the sample.')

In [ ]:
# TODO: Flip the text of the filter to make it nicer

In [ ]:
Data.Wide.unique()

In [ ]:
print('For each sample, we recorded a set of', end=' ')
if Data.Filter.unique():   
    print(" or ".join(str(value) for value in Data.Stacks.unique()),
          'stacked scans overlapping the sample height, each stack was recorded with', end=' ')
print(" or ".join(str(value) for value in Data.NumProj.unique()), 'projections of', end=' ')
for cs in Data.CamSize.unique():
    print(cs[0], end=' ')
print('x', end=' ')
for cs in Data.CamSize.unique():
    print(cs[1], end=' ')
print('pixels', end=' ')
if Data.Wide.unique():
    print('(' + " or ".join(str(value) for value in Data.Wide.unique()), 'projections stitched laterally)', end=' ')
print('at every',
      str(" or ".join(str(value) for value in Data.RotationStep.unique())) + '° over a 180° sample rotation.')

In [ ]:
Data.Exposure.mean()

In [ ]:
print('Every single projection was exposed for',
      " or ".join(str(value) for value in Data.Exposure.unique()),
      'ms,',
      " or ".join(str(value) for value in Data.Averaging.unique()),
      'projections were averaged to one to greatly reduce image noise.')

In [ ]:
log=Data.LogFile[1]

In [ ]:
print('This resulted in a scan time of approximately ', end='')
if duration(log)/3600 > 1:
    # Scan took hours
    print(timeformat(datetime.timedelta(seconds=duration(log)),
                     '{hours} hours and {minutes} minutes'), end=' ')
else:
    print(timeformat(datetime.timedelta(seconds=duration(log)),
                     '{minutes} minutes'), end=' ')
if not stacks(log) == 1:
    print('per stack and about',
          timeformat(stacks(log) * datetime.timedelta(seconds=duration(log)),
                     '{hours} hours and {minutes} minutes'), end=' ')
print('per sample', end='')
if stacks(log) == 1:
    print('.')
else:
    print(' (with', stacks(log), 'stacks).')

In [ ]:
print('In total, we scanned', Data.Stacks.sum(), 'stacks.')
print('Each stack took approximately',
      Data.Duration.mean() // 60,
      'minutes (' + str(datetime.timedelta(seconds=Data.Duration.mean())) + ')')
print('In total, we thus scanned for about', 
      timeformat(Data.Stacks.sum() *
                 datetime.timedelta(seconds=Data.Duration.mean()),
                 '{days} days, {hours} hours and {minutes} minutes.'))
print('At the MIC rate, this would have cost',
      int(round(Data.Stacks.sum() * Data.Duration.mean() / 60 / 60 * 75)),
      'CHF!')

In [ ]:
print('The projection images were then subsequently reconstructed into a 3D stack',
      'of images with',
      Data.Version.unique()[0][0],
      '(Version',
      version(log)[1] + ', Bruker microCT, Kontich Belgium)', end=' ')
if ringremoval(log):
    print('using a ring artifact correction of',
          ringremoval(log), end='')
if beamhardening(log):
    print(' and a beam hardening correction of',
          beamhardening(log),
          '%.')
else:
    print('.')
print('The whole process resulted in datasets with an isometric voxel size of',
      " or ".join(str(value) for value in Data.Voxelsize_rounded.unique()),
      'µm.')    

In [ ]:
# fulllog(log)

In [ ]:
Data.Voxelsize.mean()

In [ ]:
Data.Beamhardening